# Equalizing image hist

In [15]:
import cv2
import numpy as np
import sys
import math

img_color = cv2.imread('.\\AMI\\021_zoom_ear.jpg')
img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)

clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(10,10))
cl1 = clahe.apply(img_gray)

equalized_image = cl1

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
closing = cv2.morphologyEx(equalized_image, cv2.MORPH_CLOSE, kernel)

gaussian_filtered = cv2.GaussianBlur(closing,(9,9),0)

_, otsu = cv2.threshold(gaussian_filtered,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)


#cv2.imshow('edges otsu', otsu)
cv2.imshow('equalized', equalized_image)


contours, hierarchy = cv2.findContours(otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS )
color = cv2.cvtColor(otsu, cv2.COLOR_GRAY2BGR) 
cv2.drawContours(color, contours, -1, (0,255,0), 1)

threshold_minarea = 0.05
min_area = (img_color.shape[0]*img_color.shape[1])*threshold_minarea
threshold_maxarea = 0.5
max_area = (img_color.shape[0]*img_color.shape[1])*threshold_maxarea

if len(contours) != 0:
    for cont in contours:

        if len(cont) < 5:
            continue
        elps = cv2.fitEllipse(cont)
        (x, y), (MA, ma), angle = elps
        A = 3.1415 / 4 * MA * ma
        if math.isnan(A):
            continue
        if A<min_area or A>max_area or  x<0 or y<0 or angle < 90 :
            continue
        color = cv2.ellipse(color, elps, (0,255,0), 2)

cv2.imshow('contours2', color)

cv2.waitKey()
cv2.destroyAllWindows()